In [29]:
import  pandas               as     pd
import  pandapower           as     pp
import  numpy                as     np
import  math                 as     mt
import  cmath 
import  matplotlib.pyplot    as     plt
import  time                 as     tm
from    scipy.interpolate    import griddata
from    scipy.optimize       import minimize
from    scipy.optimize       import least_squares
from    random               import uniform
from    copy                 import deepcopy
from    itertools            import chain

In [30]:
# Set a Element Class that is common to all the elements in the grid

class Node():
    def __init__(self, node_number, node_name, vn_kv, elec_type):
        self.node_number = node_number
        self.node_name = node_name
        self.vn_kv = vn_kv
        self.elec_type = elec_type

class Line():
    def __init__(self, from_bus, to_bus, line_id, l_km, r_ohm_km, x_ohm_km, c_ohm_km, Imax_kA):
        self.from_bus = from_bus
        self.to_bus = to_bus
        self.line_id = line_id
        self.l_km = l_km
        self.r_ohm_km = r_ohm_km
        self.x_ohm_km = x_ohm_km
        self.c_ohm_km = c_ohm_km
        self.Imax_kA = Imax_kA
        
class Transformer():
    def __init__(self, from_bus, to_bus, trafo_id, sn, vk_percent, vkr_percent, pfe_kw, i0_percent, shift_degree, vector_group, tap_side, min_tap, max_tap, tap_step_degree, tap_step, tap_ps, vk0_percent, vkr0_percent, mag0_percent, mag0_rx, si0_hv_partial):
        self.from_bus = from_bus
        self.to_bus = to_bus
        self.trafo_id = trafo_id
        self.sn = sn
        self.vk_percent = vk_percent
        self.vkr_percent = vkr_percent
        self.pfe_kw = pfe_kw
        self.i0_percent = i0_percent
        self.shift_degree = shift_degree
        self.vector_group = vector_group
        self.tap_side = tap_side
        self.min_tap = min_tap
        self.max_tap = max_tap
        self.tap_step_degree = tap_step_degree
        self.tap_step = tap_step
        self.tap_ps = tap_ps
        self.vk0_percent = vk_percent
        self.vkr0_percent = vkr_percent
        self.mag0_percent = i0_percent
        self.mag0_rx = mag0_rx
        self.si0_hv_partial = si0_hv_partial

class Converter():
    def __init__(self, from_bus, to_bus, conv_id, convtype, pn, r_ohm, x_ohm, r_dc_ohm, pl_dc_mw, control_mode, control_value, eff_table):
        self.from_bus = from_bus
        self.to_bus = to_bus
        self.conv_id = conv_id
        self.convtype = convtype
        self.pn = pn
        self.r_ohm = r_ohm
        self.x_ohm = x_ohm
        self.r_dc_ohm = r_dc_ohm
        self.pl_dc_mw = pl_dc_mw
        self.control_mode = control_mode
        self.control_value = control_value
        self.eff_table = eff_table
        

class Elem:
    def __init__(self, owner, bus):
        self.owner = owner
        self.bus = bus

class Load(Elem):
    def __init__(self, owner, bus, load_id, ctype, dp_cont, vn, pn, droop):
        super().__init__(owner, bus)
        self.load_id = load_id
        self.ctype = ctype
        self.dp_cont = dp_cont
        self.vn = vn
        self.pn = pn
        self.droop = droop


class Gen(Elem):
    def __init__(self, owner, bus, gen_id, gentype, dp_cont, vn, pn, droop):
        super().__init__(owner, bus)
        self.gen_id = gen_id
        self.gentype = gentype
        self.dp_cont = dp_cont
        self.vn = vn
        self.pn = pn
        self.droop = droop

class Stor(Elem):
    def __init__(self, owner, bus, stor_id, stortype, dp_cont, ini_SOC, vn, pn, max_ener, max_p_discharge, max_p_charge, max_SOC, min_SOC, droop):
        super().__init__(owner, bus)
        self.stor_id = stor_id
        self.stortype = stortype
        self.dp_cont = dp_cont
        self.ini_SOC = ini_SOC
        self.vn = vn
        self.pn = pn
        self.max_ener = max_ener
        self.max_p_discharge = max_p_discharge
        self.max_p_charge = max_p_charge
        self.max_SOC = max_SOC
        self.min_SOC = min_SOC
        self.droop = droop


In [ ]:
# Read the Excel file
print(">Read the Excel file")
xl_file = pd.ExcelFile('C:/Users/HP/Desktop/Engenharia Eletrotécnica/Doutoramento/1º Ano/Tese/Modelação/Redes de teste/4_grid_v6.xlsx')
# xl_file = pd.ExcelFile('grid_data_buildingtest.xlsx')

# Load data from Excel sheets
print(">Load data from Excel sheets")
general_data = xl_file.parse('GeneralInfo')
node_data = xl_file.parse('NodeInfo')
line_data = xl_file.parse('LineInfo')
transformer_data = xl_file.parse('TransformerInfo')
converter_data = xl_file.parse('ConverterInfo')
load_data = xl_file.parse('LoadInfo',header = 0)#, header = [0,1,2],index_col=[0,2]) 
gen_data = xl_file.parse('GenInfo', header = 0) 
stor_data = xl_file.parse('StorInfo', header = 0) 

#print(general_data)
#print(node_data)
#print(line_data)
#print(transformer_data)
#print(converter_data)
#print(load_data)
#print(gen_data)
#print(stor_data)

In [ ]:
# Grid data for time variant PF -> Creation of the main tables of network data

def Griddata(general_data,node_data,line_data,transformer_data,converter_data,load_data,gen_data,stor_data):

    # Ensure the column names match exactly and there is no leading/trailing whitespace
    general_data.columns = general_data.columns.str.strip()
    # Define the base voltage and power from the baseValue sheet
    #print(">Define the base voltage and power from the baseValue sheet")
    sim_periods = general_data[general_data['General Information'].str.strip() == 'Simulation Periods']['Value']
    period_duration = general_data[general_data['General Information'].str.strip() == 'Periods Duration (h)']['Value']
    base_power = general_data[general_data['General Information'].str.strip() == 'Base Power (kW)']['Value']
    slack_comp = general_data[general_data['General Information'].str.strip() == 'Slack Gen Component']['Value']

    if sim_periods.empty:
        raise ValueError("The value of Number of Periods is missing.")
    elif period_duration.empty:
        raise ValueError("The value of Period Duration is missing.")
    elif base_power.empty:
        raise ValueError("Base value for Power is missing.")
    elif slack_comp.empty:
        raise ValueError("The value of Slack Component is missing.")

    base_power_kW = base_power.values[0]
    #print(base_power)

    ## Buses

    Nodelist = [Node(0,i,0,0) for i in node_data['Name']]

    for i in range(0,len(Nodelist)): 
        
        j = node_data.index[Nodelist[i].node_name == node_data['Name']]

        Nodelist[i].node_number = node_data['Node Number'].iloc[j]
    
        Nodelist[i].node_name = node_data['Name'].iloc[j]       
    
        Nodelist[i].vn_kv = node_data['Vn (kV)'].iloc[j]
    
        Nodelist[i].elec_type = node_data['Type'].iloc[j]

    ## Network equipment
    #print(">Display network data")
    #print(network_data)

    # Line equipment

    Linelist = [Line(0, 0, i, 0, 0, 0, 0, 0) for i in line_data['Component']]

    for i in range(0,len(Linelist)): 
        
        j = line_data.index[Linelist[i].line_id == line_data['Component']]

        Linelist[i].from_bus = line_data['Node_i'].iloc[j]
    
        Linelist[i].to_bus = line_data['Node_j'].iloc[j]       
    
        Linelist[i].l_km = line_data['Length_ij_km'].iloc[j]
    
        Linelist[i].r_ohm_km = line_data['Rij_ohm_km'].iloc[j]
    
        Linelist[i].x_ohm_km = line_data['Xij_ohm_km'].iloc[j]                     
    
        Linelist[i].c_ohm_km = line_data['Cij_ohm_km'].iloc[j]
    
        Linelist[i].Imax_kA = line_data['Imax_ij_kA'].iloc[j] 
        
    # Transformer equipment
    
    Trafolist = [Transformer(0, 0, i, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) for i in transformer_data['Component']]

    for i in range(0,len(Trafolist)): # This loops assume that the Load ID is coherent with the index its position (Miss make the data given to each load being dependent of its ID instead of its position in table)
        
        j = transformer_data.index[Trafolist[i].trafo_id == transformer_data['Component']]

        Trafolist[i].from_bus = transformer_data['Node_i'].iloc[j]
    
        Trafolist[i].to_bus = transformer_data['Node_j'].iloc[j]       
    
        Trafolist[i].sn = transformer_data['Sn (MVA)'].iloc[j]
    
        Trafolist[i].vk_percent = transformer_data['Short-circuit voltage (pu)'].iloc[j]
    
        Trafolist[i].vkr_percent = transformer_data['Resistive part of Short-circuit voltage (pu)'].iloc[j]
        
        Trafolist[i].pfe_kw = transformer_data['Iron losses (kW)'].iloc[j]
    
        Trafolist[i].i0_percent = transformer_data['Magnetization Current (pu)'].iloc[j]
    
        Trafolist[i].shift_degree = transformer_data['Shift degree'].iloc[j]

        Trafolist[i].vector_group = transformer_data['Vector Group'].iloc[j]

        Trafolist[i].tap_side = transformer_data['Tap Side'].iloc[j]      

        Trafolist[i].min_tap = transformer_data['Minimal Tap'].iloc[j]      
                
        Trafolist[i].max_tap = transformer_data['Maximum Tap'].iloc[j]      

        Trafolist[i].tap_step_degree = transformer_data['Tap step degree'].iloc[j]

        Trafolist[i].tap_step = transformer_data['Tap step (%)'].iloc[j]

        Trafolist[i].tap_ps = transformer_data['Tap Phase Shifter'].iloc[j]
    
        Trafolist[i].vk0_percent = transformer_data['Zero sequence short-circuit voltage (pu)'].iloc[j]    

        Trafolist[i].vk0r_percent = transformer_data['Zero sequence resistive part of short-circuit voltage (pu)'].iloc[j]     
        
        Trafolist[i].mag0_percent = transformer_data['Zero sequence of magnetization current (pu)'].iloc[j]  

        Trafolist[i].mag0_rx = transformer_data['Ratio R/X of Zero sequence of Magnetization Current'].iloc[j]    

        Trafolist[i].si0_hv_partial = transformer_data['Zero sequence short-circuit impedance distribution in HV side'].iloc[j]     
           
    # Converter equipment (In progress)

    convCarac = converter_data[['Converter ID','Unnamed: 1','Unnamed: 2','Unnamed: 3']]
    convCarac = convCarac.rename(columns={'Converter ID':convCarac['Converter ID'][0],'Unnamed: 1':convCarac['Unnamed: 1'][0],'Unnamed: 2':convCarac['Unnamed: 2'][0],'Unnamed: 3':convCarac['Unnamed: 3'][0]})
    convCarac = convCarac[1:]
    convCarac.index = range(0,len(convCarac['Component']))
    aux_idx = []
    nConvs = max(convCarac['Component'].unique())
    for i,_ in enumerate(convCarac.iterrows()):
        if (pd.isna(convCarac.iloc[i]).all()):
            aux_idx.append(i)
    convCarac = convCarac.drop(aux_idx)
    convCarac.index = range(0,convCarac.shape[0])
    nConvCarac = pd.DataFrame([convCarac['Characteristics ID'].iloc[idx] for idx,_ in enumerate(convCarac['Characteristics ID']) if idx+1 not in convCarac['Characteristics ID'] or type(convCarac['Component'].iloc[idx+1]) is int],index=convCarac['Component'].dropna().unique(),columns=['Number Caracteristics'])
    Convlist = [Converter(0,0,i,0,0,0,0,0,0,0,0,0) for i in nConvCarac.index]
    aux_nConvCarac = pd.DataFrame(np.cumsum(nConvCarac.values),index=nConvCarac.index)
    pos_Comp = np.asarray(np.where(converter_data == 'Component'))
    pos_EffCurve = np.asarray(np.where(converter_data == 'Efficiency Curve'))
    
    for i in nConvCarac.index: # This loops assume that the Converter ID is coherent with the index its position (Miss make the data given to each load being dependent of its ID instead of its position in table)
        if i-1 not in nConvCarac.index:
            j = 0
            jlim = (aux_nConvCarac.loc[i].values-1).item()
            while j<=jlim:
                if convCarac['Characteristics D.'].iloc[j]=='From Bus':
                    Convlist[i-1].from_bus = convCarac['Characteristics V.'].iloc[j]
                elif convCarac['Characteristics D.'].iloc[j]=='To Bus':
                    Convlist[i-1].to_bus = convCarac['Characteristics V.'].iloc[j]
                elif convCarac['Characteristics D.'].iloc[j]=='Converter Type':                  
                    Convlist[i-1].convtype = convCarac['Characteristics V.'].iloc[j]    
                    j_aux = 0
                    if Convlist[i-1].convtype == 'Converter AC/DC':
                        while j_aux<=jlim:
                            if convCarac['Characteristics D.'].iloc[j_aux]=='Transformer Resistance (Ohm)':
                                Convlist[i-1].r_ohm = convCarac['Characteristics V.'].iloc[j_aux]
                            elif convCarac['Characteristics D.'].iloc[j_aux]=='Transformer Reactance (Ohm)':
                                Convlist[i-1].x_ohm = convCarac['Characteristics V.'].iloc[j_aux]
                            elif convCarac['Characteristics D.'].iloc[j_aux]=='No-Load DC Power Losses (kW)':                  
                                Convlist[i-1].pl_dc_mw = convCarac['Characteristics V.'].iloc[j_aux]    
                            elif convCarac['Characteristics D.'].iloc[j_aux]=='Control Mode':                  
                                Convlist[i-1].control_mode = convCarac['Characteristics V.'].iloc[j_aux] 
                            elif convCarac['Characteristics D.'].iloc[j_aux]=='Control Value':                  
                                Convlist[i-1].control_value = convCarac['Characteristics V.'].iloc[j_aux] 
                            j_aux+=1
                elif convCarac['Characteristics D.'].iloc[j]=='Nominal Power (kW)':
                    Convlist[i-1].pn = convCarac['Characteristics V.'].iloc[j]
                j+=1
        else:
            j = (aux_nConvCarac.loc[i-1].values).item()
            jlim = (aux_nConvCarac.loc[i].values-1).item()
            while j<=jlim:
                if convCarac['Characteristics D.'].iloc[j]=='From Bus':
                    Convlist[i-1].from_bus = convCarac['Characteristics V.'].iloc[j]
                elif convCarac['Characteristics D.'].iloc[j]=='To Bus':
                    Convlist[i-1].to_bus = convCarac['Characteristics V.'].iloc[j]
                elif convCarac['Characteristics D.'].iloc[j]=='Converter Type':                  
                    Convlist[i-1].convtype = convCarac['Characteristics V.'].iloc[j]    
                    j_aux = 0
                    if Convlist[i-1].convtype == 'Converter AC/DC':
                        while j_aux<=jlim:
                            if convCarac['Characteristics D.'].iloc[j_aux]=='Transformer Resistance (Ohm)':
                                Convlist[i-1].r_ohm = convCarac['Characteristics V.'].iloc[j_aux]
                            elif convCarac['Characteristics D.'].iloc[j_aux]=='Transformer Reactance (Ohm)':
                                Convlist[i-1].x_ohm = convCarac['Characteristics V.'].iloc[j_aux]
                            elif convCarac['Characteristics D.'].iloc[j_aux]=='No-Load DC Power Losses (kW)':                  
                                Convlist[i-1].pl_dc_mw = convCarac['Characteristics V.'].iloc[j_aux]    
                            elif convCarac['Characteristics D.'].iloc[j_aux]=='Control Mode':                  
                                Convlist[i-1].control_mode = convCarac['Characteristics V.'].iloc[j_aux] 
                            elif convCarac['Characteristics D.'].iloc[j_aux]=='Control Value':                  
                                Convlist[i-1].control_value = convCarac['Characteristics V.'].iloc[j_aux] 
                            j_aux+=1
                elif convCarac['Characteristics D.'].iloc[j]=='Nominal Power (kW)':
                    Convlist[i-1].pn = convCarac['Characteristics V.'].iloc[j]
                j+=1
        
    # Get the efficency map of each converter to the list Convlist to perform the interpolations in control steps
    convEff = converter_data[converter_data.columns[int(pos_EffCurve[1]):]]
    convEff.insert(0,converter_data.columns[int(pos_Comp[1])],converter_data[converter_data.columns[int(pos_Comp[1])]])
    convEff.columns = convEff.iloc[0,:]   
    convEff = convEff[1:]  
    convEff.index = range(0,convEff.shape[0])
    aux_PF = convEff['Efficiency Curve'].dropna().unique()

    for i in range(0,len(Convlist)):
        ji = convEff.index[convEff['Component']==(i+1)].item()
        jf = ji + len(aux_PF)
        Convlist[i].eff_table = convEff.iloc[ji:jf,1:].dropna()        
    
    print(convCarac)

    # Load equipment
    #print(">Display load data")
    #print(load_data)

    # Get the caracteristics of the load components in grid from load_data DataFrame

    loadCarac = load_data[['Load ID','Unnamed: 1','Unnamed: 2','Unnamed: 3']]
    loadCarac = loadCarac.rename(columns={'Load ID':loadCarac['Load ID'][1],'Unnamed: 1':loadCarac['Unnamed: 1'][1],'Unnamed: 2':loadCarac['Unnamed: 2'][1],'Unnamed: 3':loadCarac['Unnamed: 3'][1]})
    loadCarac = loadCarac[2:]
    loadCarac.index = range(0,len(loadCarac['Component']))
    aux_idx = []
    nLoads = max(loadCarac['Component'].unique())
    for i,_ in enumerate(loadCarac.iterrows()):
        if (pd.isna(loadCarac.iloc[i]).all()):
            aux_idx.append(i)
    loadCarac = loadCarac.drop(aux_idx)
    loadCarac.index = range(0,loadCarac.shape[0])
    nLoadCarac = pd.DataFrame([loadCarac['Characteristics ID'].iloc[idx] for idx,_ in enumerate(loadCarac['Characteristics ID']) if idx+1 not in loadCarac['Characteristics ID'] or type(loadCarac['Component'].iloc[idx+1]) is int],index=loadCarac['Component'].dropna().unique(),columns=['Number Caracteristics'])
    Loadlist = [Load(0,0,i,0,0,0,0,0) for i in nLoadCarac.index]
    aux_nLoadCarac = pd.DataFrame(np.cumsum(nLoadCarac.values),index=nLoadCarac.index)

    for i in nLoadCarac.index: # This loops assume that the Load ID is coherent with the index its position (Miss make the data given to each load being dependent of its ID instead of its position in table)
        if i-1 not in nLoadCarac.index:
            j = 0
            jlim = (aux_nLoadCarac.loc[i].values-1).item()
            while j<=jlim:
                if loadCarac['Characteristics D.'].iloc[j]=='Bus Location':
                    Loadlist[i-1].bus = loadCarac['Characteristics V.'].iloc[j]
                elif loadCarac['Characteristics D.'].iloc[j]=='Droop Contribution':
                    Loadlist[i-1].dp_cont = loadCarac['Characteristics V.'].iloc[j]
                elif loadCarac['Characteristics D.'].iloc[j]=='Charge Type':                  
                    Loadlist[i-1].ctype = loadCarac['Characteristics V.'].iloc[j]                      
                elif loadCarac['Characteristics D.'].iloc[j]=='Nominal Voltage (V)':
                    Loadlist[i-1].vn = loadCarac['Characteristics V.'].iloc[j]
                elif loadCarac['Characteristics D.'].iloc[j]=='Nominal Power (kW)':
                    Loadlist[i-1].pn = loadCarac['Characteristics V.'].iloc[j]
                #    if loadCarac['Characteristics V.'].iloc[j]=='Car Charger': # Misses the verification of the specifications of specific type of loads 
                #        auxMinCurrent = 8 # Verify how to read automatically the value
                j+=1
        else:
            j = (aux_nLoadCarac.loc[i-1].values).item()
            jlim = (aux_nLoadCarac.loc[i].values-1).item()
            while j<=jlim:
                if loadCarac['Characteristics D.'].iloc[j]=='Bus Location':
                    Loadlist[i-1].bus = loadCarac['Characteristics V.'].iloc[j]
                elif loadCarac['Characteristics D.'].iloc[j]=='Droop Contribution':
                    Loadlist[i-1].dp_cont = loadCarac['Characteristics V.'].iloc[j]
                elif loadCarac['Characteristics D.'].iloc[j]=='Charge Type':                   
                    Loadlist[i-1].ctype = loadCarac['Characteristics V.'].iloc[j]                     
                elif loadCarac['Characteristics D.'].iloc[j]=='Nominal Voltage (V)':
                    Loadlist[i-1].vn = loadCarac['Characteristics V.'].iloc[j]
                elif loadCarac['Characteristics D.'].iloc[j]=='Nominal Power (kW)':
                    Loadlist[i-1].pn = loadCarac['Characteristics V.'].iloc[j]
                j+=1


    #print(loadCarac)

    # Get a matrix of all voltages and powers points of Current/OS voltage/power droop curves from load_data DataFrame to perform the interpolations

    aux_namCol = ['Load ID','Unnamed: 4','Total Time (h)']
    for i,j in enumerate(load_data.columns):
        if type(j) is int:                       
            aux_namCol.append(j)                    # Gives the correct column names to search the droop information in time in load_data DataFrame 


    loadDroop = load_data[aux_namCol] 
    loadDroop = loadDroop.rename(columns={'Load ID':loadDroop['Load ID'][1],'Unnamed: 4':loadDroop['Unnamed: 4'][1],'Total Time (h)':loadDroop['Total Time (h)'][1]})
    loadDroop = loadDroop[2:]
    maxDroopsLoad = max(loadDroop['Droop ID'].unique())
    loadDroop.index = range(0,loadDroop.shape[0])

    #print(loadDroop)


    for i in nLoadCarac.index:
        aux_indx = (np.array(loadDroop.index[loadDroop['Component']==i].tolist())).item()
        if aux_indx is not None:
            for j in range(0,len(Loadlist)):
                if Loadlist[j].load_id == i:
                    Loadlist[j].droop = loadDroop.loc[aux_indx:aux_indx+maxDroopsLoad-1]

    # This automatic verification of the name pairs of V,P allows to use several numerations of the voltage droop control curves, for exemple, (V1, P1) or (Va, Pa) or (VA,PA)

    for i in range(0,len(Loadlist)):                # Sorting process (load by load)
        
        auxV = pd.DataFrame(Loadlist[i].droop.loc[Loadlist[i].droop.index[Loadlist[i].droop['Droop'].str.contains('V')],'Droop'],index=Loadlist[i].droop.index[Loadlist[i].droop['Droop'].str.contains('V')].tolist())      # List of the names of the voltage points of the Current/OS voltage droop control curves of each load
        auxV.sort_values(by='Droop')                                                                                                                                                                                        # This command guarrantees that the voltage names are sorted

        auxP = pd.DataFrame(Loadlist[i].droop.loc[Loadlist[i].droop.index[Loadlist[i].droop['Droop'].str.contains(r'P\d+', regex=True)],'Droop'],index=Loadlist[i].droop.index[Loadlist[i].droop['Droop'].str.contains(r'P\d+', regex=True)].tolist())      # List of the names of the voltage points of the Current/OS voltage droop control curves of each load
        auxP.sort_values(by='Droop')                                                                                                                                                                                        # This command guarrantees that the voltage names are sorted

        auxVP = list(zip(auxV.index,auxP.index))    # List with tuples with the indexes corresponding to the names of the "coordinates" of the points of the Current/OS voltage droop control curves of each load

        for k in Loadlist[i].droop.columns:
            if type(k) is int:
                droop_list = []
                for j,_ in enumerate(auxVP): 
                    droop_list.append(Loadlist[i].droop.loc[np.array(list(auxVP[j])),k].values)  
                df = pd.DataFrame(droop_list, columns=['x','y'])
                df_sorted = df.sort_values(by=['x','y'])
                Loadlist[i].droop.loc[auxV.index,k] = df_sorted['x'].values
                Loadlist[i].droop.loc[auxP.index,k] = df_sorted['y'].values

    # Set the Load Class to characterize each load

    # Generator equipment
    #print(">Display generator data")
    #print(gen_data)

    # Get the caracteristics of the generator components in grid from gen_data DataFrame

    genCarac = gen_data[['Gen ID','Unnamed: 1','Unnamed: 2','Unnamed: 3']]
    genCarac = genCarac.rename(columns={'Gen ID':genCarac['Gen ID'][1],'Unnamed: 1':genCarac['Unnamed: 1'][1],'Unnamed: 2':genCarac['Unnamed: 2'][1],'Unnamed: 3':genCarac['Unnamed: 3'][1]})
    genCarac = genCarac[2:]
    genCarac.index = range(0,len(genCarac['Component']))
    aux_idx = []
    nGens = max(genCarac['Component'].unique())
    for i,_ in enumerate(genCarac.iterrows()):
        if (pd.isna(genCarac.iloc[i]).all()):
            aux_idx.append(i)
    genCarac = genCarac.drop(aux_idx)
    genCarac.index = range(0,genCarac.shape[0])
    nGenCarac = pd.DataFrame([genCarac['Characteristics ID'].iloc[idx] for idx,_ in enumerate(genCarac['Characteristics ID']) if idx+1 not in genCarac['Characteristics ID'] or type(genCarac['Component'].iloc[idx+1]) is int],index=genCarac['Component'].dropna().unique(),columns=['Number Caracteristics'])
    Genlist = [Gen(0,0,i,0,0,0,0,0) for i in nGenCarac.index]
    aux_nGenCarac = pd.DataFrame(np.cumsum(nGenCarac.values),index=nGenCarac.index)
        
    for i in nGenCarac.index: 
        if i-1 not in nGenCarac.index:
            j=0
            jlim = (aux_nGenCarac.loc[i].values-1).item()
            while j<=jlim:
                if genCarac['Characteristics D.'].iloc[j]=='Bus Location':
                    Genlist[int(i-1)].bus = genCarac['Characteristics V.'].iloc[j]
                elif genCarac['Characteristics D.'].iloc[j]=='Droop Contribution':
                    Genlist[int(i-1)].dp_cont = genCarac['Characteristics V.'].iloc[j]
                elif genCarac['Characteristics D.'].iloc[j]=='Gen Type':                    
                    Genlist[i-1].gentype = genCarac['Characteristics V.'].iloc[j]                     
                elif genCarac['Characteristics D.'].iloc[j]=='Nominal Voltage (V)':
                    Genlist[i-1].vn = genCarac['Characteristics V.'].iloc[j]
                elif genCarac['Characteristics D.'].iloc[j]=='Nominal Power (kW)':
                    Genlist[i-1].pn = genCarac['Characteristics V.'].iloc[j]
                j+=1
        else:
            j = (aux_nGenCarac.loc[i-1].values).item()
            jlim = (aux_nGenCarac.loc[i].values-1).item()
            while j<=jlim:
                if genCarac['Characteristics D.'].iloc[j]=='Bus Location':
                    Genlist[i-1].bus = genCarac['Characteristics V.'].iloc[j]
                elif genCarac['Characteristics D.'].iloc[j]=='Droop Contribution':
                    Genlist[i-1].dp_cont = genCarac['Characteristics V.'].iloc[j]
                elif genCarac['Characteristics D.'].iloc[j]=='Gen Type':                    
                    Genlist[i-1].gentype = genCarac['Characteristics V.'].iloc[j]                    
                elif genCarac['Characteristics D.'].iloc[j]=='Nominal Voltage (V)':
                    Genlist[i-1].vn = genCarac['Characteristics V.'].iloc[j]
                elif genCarac['Characteristics D.'].iloc[j]=='Nominal Power (kW)':
                    Genlist[i-1].pn = genCarac['Characteristics V.'].iloc[j]
                j+=1

    #print(genCarac)

    # Get a matrix of all voltages and powers points of Current/OS voltage/power droop curves from gen_data DataFrame to perform the interpolations

    aux_namCol = ['Gen ID','Unnamed: 4','Total Time (h)']
    for i,j in enumerate(gen_data.columns):
        if type(j) is int:                       
            aux_namCol.append(j)                    # Gives the correct column names to search the droop information in time in gen_data DataFrame 


    genDroop = gen_data[aux_namCol] 
    genDroop = genDroop.rename(columns={'Gen ID':genDroop['Gen ID'][1],'Unnamed: 4':genDroop['Unnamed: 4'][1],'Total Time (h)':genDroop['Total Time (h)'][1]})
    genDroop = genDroop[2:]
    maxDroopsGen = max(genDroop['Droop ID'].unique())
    genDroop.index = range(0,genDroop.shape[0])

    #print(genDroop)

    for i in nGenCarac.index:
        aux_indx = (np.array(genDroop.index[genDroop['Component']==i].tolist())).item()
        if aux_indx is not None:
            for j in range(0,len(Genlist)):
                if Genlist[j].gen_id == i:
                    Genlist[j].droop = genDroop.loc[aux_indx:aux_indx+maxDroopsGen-1]

    for i in range(0,len(Genlist)):                # Sorting process (generator by generator)
        
        auxV = pd.DataFrame(Genlist[i].droop.loc[Genlist[i].droop.index[Genlist[i].droop['Droop'].str.contains('V')],'Droop'],index=Genlist[i].droop.index[Genlist[i].droop['Droop'].str.contains('V')].tolist())      # List of the names of the voltage points of the Current/OS voltage droop control curves of each generator
        auxV.sort_values(by='Droop')                                                                                                                                                                                        # This command guarrantees that the voltage names are sorted

        auxP = pd.DataFrame(Genlist[i].droop.loc[Genlist[i].droop.index[Genlist[i].droop['Droop'].str.contains(r'P\d+', regex=True)],'Droop'],index=Genlist[i].droop.index[Genlist[i].droop['Droop'].str.contains(r'P\d+', regex=True)].tolist())      # List of the names of the voltage points of the Current/OS voltage droop control curves of each generator
        auxP.sort_values(by='Droop')                                                                                                                                                                                        # This command guarrantees that the voltage names are sorted

        auxVP = list(zip(auxV.index,auxP.index))    # List with tuples with the indexes corresponding to the names of the "coordinates" of the points of the Current/OS voltage droop control curves of each generator

        for k in Genlist[i].droop.columns:
            if type(k) is int:
                droop_list = []
                for j,_ in enumerate(auxVP): 
                    droop_list.append(Genlist[i].droop.loc[np.array(list(auxVP[j])),k].values)  
                df = pd.DataFrame(droop_list, columns=['x','y'])
                df_sorted = df.sort_values(by=['x','y'])
                Genlist[i].droop.loc[auxV.index,k] = df_sorted['x'].values
                Genlist[i].droop.loc[auxP.index,k] = df_sorted['y'].values


    #print(Genlist)

    # Storage equipment 
    #print(">Display Storage data") 
    #print(stor_data)

    # Get the caracteristics of the storage components in grid from stor_data DataFrame

    storCarac = stor_data[['Storage ID','Unnamed: 1','Unnamed: 2','Unnamed: 3']]
    storCarac = storCarac.rename(columns={'Storage ID':storCarac['Storage ID'][1],'Unnamed: 1':storCarac['Unnamed: 1'][1],'Unnamed: 2':storCarac['Unnamed: 2'][1],'Unnamed: 3':storCarac['Unnamed: 3'][1]})
    storCarac = storCarac[2:]
    storCarac.index = range(0,len(storCarac['Component']))
    aux_idx = []
    nStor = max(storCarac['Component'].unique())
    for i,_ in enumerate(storCarac.iterrows()):
        if (pd.isna(storCarac.iloc[i]).all()):
            aux_idx.append(i)
    storCarac = storCarac.drop(aux_idx)
    storCarac.index = range(0,storCarac.shape[0])
    nStorCarac = pd.DataFrame([storCarac['Characteristics ID'].iloc[idx] for idx,_ in enumerate(storCarac['Characteristics ID']) if idx+1 not in storCarac['Characteristics ID'] or type(storCarac['Component'].iloc[idx+1]) is int],index=storCarac['Component'].dropna().unique(),columns=['Number Caracteristics'])
    Storlist = [Stor(0, 0, i, 0, 0, 0, 0, 0, 0, 0, 0, 0.00001, 0, 0) for i in nStorCarac.index]
    aux_nStorCarac = pd.DataFrame(np.cumsum(nStorCarac.values),index=nStorCarac.index)
        
    for i in nStorCarac.index: 
        if i-1 not in nStorCarac.index:
            j=0
            jlim = aux_nStorCarac.loc[i].values-1
            while j<=jlim:
                if storCarac['Characteristics D.'].iloc[j]=='Bus Location':
                    Storlist[int(i-1)].bus = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Droop Contribution':
                    Storlist[i-1].dp_cont = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Initial SOC (%)':
                    Storlist[int(i-1)].ini_SOC = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Storage Type':                    
                    Storlist[i-1].gentype = storCarac['Characteristics V.'].iloc[j]                     
                elif storCarac['Characteristics D.'].iloc[j]=='Nominal Voltage (V)':
                    Storlist[i-1].vn = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Nominal Power (kW)':
                    Storlist[i-1].pn = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Maximum Energy (kWh)':
                    Storlist[i-1].max_ener = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Maximum Power Delivery (kW)':
                    Storlist[i-1].max_p_discharge = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Maximum Power Charge (kW)':
                    Storlist[i-1].max_p_charge = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Upper SOC limit (%)':
                    Storlist[i-1].max_SOC = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Lower SOC limit (%)':
                    Storlist[i-1].min_SOC = storCarac['Characteristics V.'].iloc[j]
                j+=1
        else:
            j=int(aux_nStorCarac.loc[i-1].values)
            jlim = int(aux_nStorCarac.loc[i].values-1)
            while j<=jlim:
                if storCarac['Characteristics D.'].iloc[j]=='Bus Location':
                    Storlist[int(i-1)].bus = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Droop Contribution':
                    Storlist[i-1].dp_cont = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Initial SOC (%)':
                    Storlist[int(i-1)].ini_SOC = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Storage Type':                    
                    Storlist[i-1].gentype = storCarac['Characteristics V.'].iloc[j]                     
                elif storCarac['Characteristics D.'].iloc[j]=='Nominal Voltage (V)':
                    Storlist[i-1].vn = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Nominal Power (kW)':
                    Storlist[i-1].pn = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Maximum Energy (kWh)':
                    Storlist[i-1].max_ener = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Maximum Power Delivery (kW)':
                    Storlist[i-1].max_p_discharge = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Maximum Power Charge (kW)':
                    Storlist[i-1].max_p_charge = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Upper SOC limit (%)':
                    Storlist[i-1].max_SOC = storCarac['Characteristics V.'].iloc[j]
                elif storCarac['Characteristics D.'].iloc[j]=='Lower SOC limit (%)':
                    Storlist[i-1].min_SOC = storCarac['Characteristics V.'].iloc[j]
                j+=1

    #print(storCarac)

    # Get a matrix of all voltages and powers points of Current/OS voltage/power droop curves from stor_data DataFrame to perform the interpolations

        # Get a matrix of all voltages and powers points of Current/OS voltage/power droop curves from gen_data DataFrame to perform the interpolations

    aux_namCol = ['Storage ID','Unnamed: 4','Total Time (h)']
    for i,j in enumerate(stor_data.columns):
        if type(j) is int:                       
            aux_namCol.append(j)                    # Gives the correct column names to search the droop information in time in gen_data DataFrame 


    storDroop = stor_data[aux_namCol] 
    storDroop = storDroop.rename(columns={'Storage ID':storDroop['Storage ID'][1],'Unnamed: 4':storDroop['Unnamed: 4'][1],'Total Time (h)':storDroop['Total Time (h)'][1]})
    storDroop = storDroop[2:]
    maxDroopsStor = max(storDroop['Droop ID'].unique())
    storDroop.index = range(0,storDroop.shape[0])

    #print(genDroop)

    for i in nStorCarac.index:
        aux_indx = (np.array(storDroop.index[storDroop['Component']==i].tolist())).item()
        if aux_indx is not None:
            for j in range(0,len(Storlist)):
                if Storlist[j].stor_id == i:
                    Storlist[j].droop = storDroop.loc[aux_indx:aux_indx+maxDroopsStor-1]

    for i in range(0,len(Storlist)):                # Sorting process (generator by generator)
        
        auxV = pd.DataFrame(Storlist[i].droop.loc[Storlist[i].droop.index[Storlist[i].droop['Droop'].str.contains('V')],'Droop'],index=Storlist[i].droop.index[Storlist[i].droop['Droop'].str.contains('V')].tolist())      # List of the names of the voltage points of the Current/OS voltage droop control curves of each generator
        auxV.sort_values(by='Droop')                                                                                                                                                                                        # This command guarrantees that the voltage names are sorted

        auxP = pd.DataFrame(Storlist[i].droop.loc[Storlist[i].droop.index[Storlist[i].droop['Droop'].str.contains(r'P\d+', regex=True)],'Droop'],index=Storlist[i].droop.index[Storlist[i].droop['Droop'].str.contains(r'P\d+', regex=True)].tolist())      # List of the names of the voltage points of the Current/OS voltage droop control curves of each generator
        auxP.sort_values(by='Droop')                                                                                                                                                                                        # This command guarrantees that the voltage names are sorted

        auxVP = list(zip(auxV.index,auxP.index))    # List with tuples with the indexes corresponding to the names of the "coordinates" of the points of the Current/OS voltage droop control curves of each generator

        for k in Storlist[i].droop.columns:
            if type(k) is int:
                droop_list = []
                for j,_ in enumerate(auxVP): 
                    droop_list.append(Storlist[i].droop.loc[np.array(list(auxVP[j])),k].values)  
                df = pd.DataFrame(droop_list, columns=['x','y'])
                df_sorted = df.sort_values(by=['x','y'])
                Storlist[i].droop.loc[auxV.index,k] = df_sorted['x'].values
                Storlist[i].droop.loc[auxP.index,k] = df_sorted['y'].values


    #print(Storlist)

    ########## NETWORK CREATION ###########

    # Create an empty pandapower network
    #print(">Create an empty pandapower network")
    net = pp.create_empty_network(name='', f_hz=50.0, sn_mva=base_power.item()/1e3)
    #print(net)

    
    # Create buses
    #print(">Create buses") 
    
    buses = {}
    
    for i in range(0,len(Nodelist)):
        buses[Nodelist[i].node_number.item()] = pp.create_bus(net, vn_kv=Nodelist[i].vn_kv.item(), name=f"Bus {Nodelist[i].node_name.item()}") # Convert from V to kV (Important to PF convergence)
    
    #print(buses)

    for i in range(0,len(Linelist)):

        pp.create_line_from_parameters(net,
                                    name='Line ' + str(Linelist[i].line_id),
                                    from_bus=buses[Linelist[i].from_bus.item()],
                                    to_bus=buses[Linelist[i].to_bus.item()],
                                    length_km=Linelist[i].l_km.item(),
                                    r_ohm_per_km=Linelist[i].r_ohm_km.item(),
                                    x_ohm_per_km=Linelist[i].x_ohm_km.item(),
                                    c_nf_per_km=Linelist[i].c_ohm_km.item(),
                                    max_i_ka=Linelist[i].Imax_kA.item())

    # Create transformers
    #print(">Create transformers") 

    for i in range(0,len(Trafolist)):

        aux_fbus_idx = 0
        aux_tbus_idx = 0

        for j in range(0,len(Nodelist)):
            if Nodelist[j].node_number.item() == Trafolist[i].from_bus.item():
                aux_fbus_idx = j
            if Nodelist[j].node_number.item() == Trafolist[i].to_bus.item():
                aux_tbus_idx = j

        pp.create_std_type(net, {"sn_mva": Trafolist[i].sn.item(),
            "vn_hv_kv": Nodelist[aux_fbus_idx].vn_kv.item(),
            "vn_lv_kv": Nodelist[aux_tbus_idx].vn_kv.item(),
            "vk_percent": Trafolist[i].vk_percent.item(),
            "vkr_percent": Trafolist[i].vkr_percent.item(),
            "pfe_kw": Trafolist[i].pfe_kw.item(),
            "i0_percent": Trafolist[i].i0_percent.item(),
            "shift_degree": Trafolist[i].shift_degree.item(),
            "vector_group": str(Trafolist[i].vector_group.item()),
            "tap_side": str(Trafolist[i].tap_side.item()),
            "tap_neutral": 0, # Miss in Excel file
            "tap_min": Trafolist[i].min_tap.item(),
            "tap_max": Trafolist[i].max_tap.item(),
            "tap_step_degree": Trafolist[i].tap_step_degree.item(),
            "tap_step_percent": Trafolist[i].tap_step.item(),
            "tap_phase_shifter": Trafolist[i].tap_ps.item()=='True',
            "vk0_percent": Trafolist[i].vk0_percent.item(),
            "vkr0_percent": Trafolist[i].vkr0_percent,
            "mag0_percent": Trafolist[i].mag0_percent.item(),
            "mag0_rx": Trafolist[i].mag0_rx.item(),
            "si0_hv_partial": Trafolist[i].si0_hv_partial.item(),}, name='Transformer', element="trafo")

        pp.create_transformer(net, hv_bus = buses[Trafolist[i].from_bus.item()], lv_bus = buses[Trafolist[i].to_bus.item()], name = 'Transformer ' + str(Trafolist[i].trafo_id), std_type = 'Transformer')

    # Create converters
    #print(">Create converters") 

    for i in range(0,len(Convlist)):

        if Convlist[i].convtype == 'Converter DC/DC':

            # For the DC/DC converters we assume a analogy with three phase transformers (pandapower model -> transformer model)

            aux_fbus_idx = 0
            aux_tbus_idx = 0

            for j in range(0,len(Nodelist)):
                if Nodelist[j].node_number.item() == Convlist[i].from_bus:
                    aux_fbus_idx = j
                if Nodelist[j].node_number.item() == Convlist[i].to_bus:
                    aux_tbus_idx = j

            pp.create_std_type(net, {"sn_mva": Convlist[i].pn,
                "vn_hv_kv": Nodelist[aux_fbus_idx].vn_kv.item(),
                "vn_lv_kv": Nodelist[aux_tbus_idx].vn_kv.item(),
                "vk_percent": 0.01,
                "vkr_percent": 0.01,
                "pfe_kw": 0,
                "i0_percent": 0,
                "shift_degree": 0,
                "vector_group": 'Yyn',
                "tap_side": 'lv',
                "tap_neutral": 0, 
                "tap_min":-2,
                "tap_max": 2,
                "tap_step_degree": 0,
                "tap_step_percent": 2.5,
                "tap_phase_shifter": False,
                "vk0_percent": 0,
                "vkr0_percent": 0,
                "mag0_percent": 0,
                "mag0_rx": 0,
                "si0_hv_partial": 0,}, name='Converter DC/DC', element="trafo")

            pp.create_transformer(net, hv_bus = buses[Convlist[i].from_bus], lv_bus = buses[Convlist[i].to_bus], name = 'Converter DC/DC ' + str(Convlist[i].conv_id), std_type = 'Converter DC/DC')

        elif Convlist[i].convtype == 'Converter AC/DC': 

            # Also for the AC/DC converters we assume a two network element model (pandapower models -> transformer model + ward model)

            aux_fbus_idx = 0
            aux_tbus_idx = 0

            for j in range(0,len(Nodelist)):
                
                # Transformer buses automatic finding
                
                if Nodelist[j].node_number.item() == Convlist[i].from_bus:
                    aux_fbus_idx = j
                if Nodelist[j].node_number.item() == Convlist[i].to_bus:
                    aux_tbus_idx = j
                
                # Ward bus automatic finding
                
                if Nodelist[j].elec_type.item() == 'AC':
                    aux_ward_idx = j
                    if Nodelist[j].node_number.item() == Convlist[i].from_bus:
                        aux_ward_bus_idx = Convlist[i].from_bus
                    elif Nodelist[j].node_number.item() == Convlist[i].to_bus:
                        aux_ward_bus_idx = Convlist[i].to_bus


            pp.create_std_type(net, {"sn_mva": Convlist[i].pn,
                "vn_hv_kv": Nodelist[aux_fbus_idx].vn_kv.item(),
                "vn_lv_kv": Nodelist[aux_tbus_idx].vn_kv.item(),
                "vk_percent": 0.01,
                "vkr_percent": 0.009,
                "pfe_kw": 0,
                "i0_percent": 0,
                "shift_degree": 0,
                "vector_group": 'Yyn',
                "tap_side": 'lv',
                "tap_neutral": 0, 
                "tap_min":-2,
                "tap_max": 2,
                "tap_step_degree": 0,
                "tap_step_percent": 2.5,
                "tap_phase_shifter": False,
                "vk0_percent": 0,
                "vkr0_percent": 0,
                "mag0_percent": 0,
                "mag0_rx": 0,
                "si0_hv_partial": 0,}, name='Converter AC/DC', element="trafo")

            pp.create_transformer(net, hv_bus = buses[Convlist[i].from_bus], lv_bus = buses[Convlist[i].to_bus], name = 'Converter AC/DC ' + str(Convlist[i].conv_id), std_type = 'Converter AC/DC')    # Creation of transformer element

            pp.create_ward(net, buses[aux_ward_bus_idx], 0, 0, 0, 0, name = 'Converter AC/DC ' + str(Convlist[i].conv_id))  # Creation of ward element and corresponding initialization of reactive power in zero

            #pp.create_vsc(net,                              # It is dependent from vsc development from pandapower
            #    bus = buses[Convlist[i].from_bus],
            #    bus_dc = buses[Convlist[i].to_bus],
            #    r_ohm = Convlist[i].r_ohm.item(),
            #    x_ohm = Convlist[i].x_ohm.item(),
            #    r_dc_ohm = Convlist[i].r_dc_ohm.item(),
            #    pl_dc_mw = Convlist[i].pl_dc_mw.item(),
            #    control_mode_ac = str(Convlist[i].control_mode_ac.item()),  
            #    control_value_ac = Convlist[i].control_value_ac.item(), 
            #    control_mode_dc = str(Convlist[i].control_mode_dc.item()), 
            #    control_value_dc = Convlist[i].control_value_dc.item(),
            #    name='Converter AC/DC ' + str(Convlist[i].conv_id))

    # Create loads, gens and storage (in progress)
    #print(">Create loads, generators and storage")

    # Load equipment
    #print(">Create loads")

    for i in range(0,len(Loadlist)):
        auxV = Loadlist[i].droop.index[Loadlist[i].droop['Droop'].str.contains('V')]
        auxP = Loadlist[i].droop.index[Loadlist[i].droop['Droop'].str.contains(r'P\d+', regex=True)]
        auxPS = Loadlist[i].droop.index[Loadlist[i].droop['Droop'].str.contains(r'P[A-Za-z]+', regex=True)]
        auxQS = Loadlist[i].droop.index[Loadlist[i].droop['Droop'].str.contains(r'Q[A-Za-z]+', regex=True)]
        auxT = min([i for i in Loadlist[i].droop.columns if type(i) is int])
        for j in range(0,len(Nodelist)):
            if Nodelist[j].node_number.item() == Loadlist[i].bus:
                aux_electype = Nodelist[j].elec_type.item()
        if aux_electype == 'DC':
            if Loadlist[i].dp_cont == 1:
                pdroop = np.interp(1,Loadlist[i].droop.loc[auxV,auxT].values,Loadlist[i].droop.loc[auxP,auxT].values)
                pset = Loadlist[i].droop.loc[auxPS,auxT].values
                pload = 0
                if abs(pset) > abs(pdroop): # Verification if setpoint of power is higher than droop curve
                    pload = pdroop
                else:
                    pload = pset

                pp.create_load(net,
                            name=Loadlist[i].load_id,
                            bus=buses[Loadlist[i].bus], # The interpolation is the initial droop adjustment assuming a flat start in PF solving
                            p_mw=pload*Loadlist[i].pn/1000,  # Convert kW to MW 
                            q_mvar=0)  # assuming zero reactive power
            else:
                pp.create_load(net,
                            name=Loadlist[i].load_id,
                            bus=buses[Loadlist[i].bus], # The interpolation is the initial droop adjustment assuming a flat start in PF solving
                            p_mw=(Loadlist[i].droop.loc[auxPS,auxT])*Loadlist[i].pn/1000,  # Convert kW to MW, it is assumed the maximum power point as constant power load situation (dp_cont variable equal to 0)
                            q_mvar=0)  # assuming zero reactive power
        elif aux_electype == 'AC':
            pp.create_load(net,
                            name=Loadlist[i].load_id,
                            bus=buses[Loadlist[i].bus], # The interpolation is the initial droop adjustment assuming a flat start in PF solving
                            p_mw=(Loadlist[i].droop.loc[auxPS,auxT])*Loadlist[i].pn/1000,  # Convert kW to MW, it is assumed the maximum power point as constant power load situation (dp_cont variable equal to 0)
                            q_mvar=(Loadlist[i].droop.loc[auxQS,auxT])*Loadlist[i].pn/1000)  # assuming zero reactive power
        
    # Generator equipment
    #print(">Create generators")

    for i in range(0,len(Genlist)):
        auxV = Genlist[i].droop.index[Genlist[i].droop['Droop'].str.contains('V')]
        auxP = Genlist[i].droop.index[Genlist[i].droop['Droop'].str.contains(r'P\d+', regex=True)]
        auxPS = Genlist[i].droop.index[Genlist[i].droop['Droop'].str.contains(r'P[A-Za-z]+', regex=True)]
        auxQS = Genlist[i].droop.index[Genlist[i].droop['Droop'].str.contains(r'Q[A-Za-z]+', regex=True)]
        auxT = min([i for i in Genlist[i].droop.columns if type(i) is int])
        for j in range(0,len(Nodelist)):
            if Nodelist[j].node_number.item() == Genlist[i].bus:
                aux_electype = Nodelist[j].elec_type.item()
        if aux_electype == 'DC':
            if Genlist[i].gen_id == slack_comp.values:
                if Genlist[i].dp_cont == 1:
                    pdroop = np.interp(1,Genlist[i].droop.loc[auxV,auxT].values,Genlist[i].droop.loc[auxP,auxT].values)
                    pset = Genlist[i].droop.loc[auxPS,auxT].values
                    pload = 0

                    if abs(pset) > abs(pdroop): # Verification if setpoint of power is higher than droop curve
                        pload = pdroop
                    else:
                        pload = pset

                    pp.create_gen(net,
                            name=Genlist[i].gen_id,
                            bus=buses[Genlist[i].bus],      # The interpolation is the initial droop adjustment assuming a flat start in PF solving
                            p_mw=pset*Genlist[i].pn/1000,   # Convert kW to MW 
                            vm_pu=1,                        # assuming a voltage setpoint of 1 (Can be an improvement to make by saying that one of the caracteristics of the gen is the voltage setpoint)
                            slack=True)  
                else:
                    pp.create_gen(net,
                            name=Genlist[i].gen_id,
                            bus=buses[Genlist[i].bus], 
                            p_mw=(Genlist[i].droop.loc[auxPS,auxT])*Genlist[i].pn/1000,  # Convert kW to MW, it is assumed the maximum power point as constant power gen situation (dp_cont variable equal to 0)
                            vm_pu=1,
                            slack=True)      
            elif Genlist[i].dp_cont == 1: # Non-slack generators (These generators must be static ones since it is only known the P or by its output power or by its droop curve)
                pdroop = np.interp(1,Genlist[i].droop.loc[auxV,auxT].values,Genlist[i].droop.loc[auxP,auxT].values)
                pset = Genlist[i].droop.loc[auxPS,auxT].values
                pload = 0
                if abs(pset) > abs(pdroop): # Verification if setpoint of power is higher than droop curve
                    pload = pdroop
                else:
                    pload = pset
                pp.create_sgen(net,
                            name=Genlist[i].gen_id,
                            bus=buses[Genlist[i].bus], 
                            p_mw=np.interp(1,Genlist[i].droop.loc[auxV,auxT].values,Genlist[i].droop.loc[auxP,auxT].values)*Genlist[i].pn/1000)  
            else:
                pp.create_sgen(net,
                            name=Genlist[i].gen_id,
                            bus=buses[Genlist[i].bus], 
                            p_mw=(Genlist[i].droop.loc[auxPS,auxT])*Genlist[i].pn/1000)   
        elif aux_electype == 'AC':
            if Genlist[i].gen_id == slack_comp.values:
                pp.create_gen(net,
                            name=Genlist[i].gen_id,
                            bus=buses[Genlist[i].bus], # The interpolation is the initial droop adjustment assuming a flat start in PF solving
                            p_mw=(Genlist[i].droop.loc[auxPS,auxT])*Genlist[i].pn/1000,  # Convert kW to MW, it is assumed the maximum power point as constant power load situation (dp_cont variable equal to 0)
                            vm_pu=1,
                            max_q_mvar=(Genlist[i].droop.loc[auxQS,auxT])*Genlist[i].pn/1000,
                            slack = True)  
            else:
                pp.create_sgen(net,
                            name=Genlist[i].gen_id,
                            bus=buses[Genlist[i].bus], 
                            p_mw=(Genlist[i].droop.loc[auxPS,auxT])*Genlist[i].pn/1000,
                            max_q_mvar=(Genlist[i].droop.loc[auxQS,auxT])*Genlist[i].pn/1000)   

    # Storage equipment
    #print(">Create BESS")  # Misses how to consider a BESS element to be the slack component

    for i in range(0,len(Storlist)):
        auxV = Storlist[i].droop.index[Storlist[i].droop['Droop'].str.contains('V')]
        auxP = Storlist[i].droop.index[Storlist[i].droop['Droop'].str.contains(r'P\d+', regex=True)]
        auxPS = Storlist[i].droop.index[Storlist[i].droop['Droop'].str.contains(r'P[A-Za-z]+', regex=True)]
        auxQS = Storlist[i].droop.index[Storlist[i].droop['Droop'].str.contains(r'Q[A-Za-z]+', regex=True)]
        auxT = min([i for i in Storlist[i].droop.columns if type(i) is int])
        for j in range(0,len(Nodelist)):
            if Nodelist[j].node_number.item() == Storlist[i].bus:
                aux_electype = Nodelist[j].elec_type.item()
        if aux_electype == 'DC':
            if Storlist[i].dp_cont == 1:
                pdroop = np.interp(1,Storlist[i].droop.loc[auxV,auxT].values,Storlist[i].droop.loc[auxP,auxT].values)
                pset = Storlist[i].droop.loc[auxPS,auxT].values
                pload = 0
                if abs(pset) > abs(pdroop): # Verification if setpoint of power is higher than droop curve
                    pload = pdroop
                else:
                    pload = pset

                pp.create_storage(net,
                            name=Storlist[i].stor_id,
                            bus=buses[Storlist[i].bus], # The interpolation is the initial droop adjustment assuming a flat start in PF solving
                            p_mw=pload*Storlist[i].pn/1000,  # Convert kW to MW 
                            q_mvar=0,    # assuming zero reactive power (DC)
                            max_e_mwh=Storlist[i].max_ener/1000, # Convert from kWh to MWh
                            soc_percent=Storlist[i].ini_SOC)   
            else:
                pp.create_storage(net,
                            name=Storlist[i].stor_id,
                            bus=buses[Storlist[i].bus], # The interpolation is the initial droop adjustment assuming a flat start in PF solving
                            p_mw=(Storlist[i].droop.loc[auxPS,auxT])*Storlist[i].pn/1000,  # Convert kW to MW, it is assumed the maximum power point as constant power load situation (dp_cont variable equal to 0)
                            q_mvar=0,
                            max_e_mwh=Storlist[i].max_ener/1000, # Convert from kWh to MWh
                            soc_percent=Storlist[i].ini_SOC)
        elif aux_electype == 'AC':
            pp.create_storage(net,
                            name=Storlist[i].stor_id,
                            bus=buses[Storlist[i].bus], # The interpolation is the initial droop adjustment assuming a flat start in PF solving
                            p_mw=(Storlist[i].droop.loc[auxPS,auxT])*Storlist[i].pn/1000,  # Convert kW to MW, it is assumed the maximum power point as constant power load situation (dp_cont variable equal to 0)
                            q_mvar=(Storlist[i].droop.loc[auxQS,auxT])*Storlist[i].pn/1000,  # assuming zero reactive power
                            max_e_mwh=Storlist[i].max_ener/1000, # Convert from kWh to MWh
                            soc_percent=Storlist[i].ini_SOC)

    return buses, base_power, Nodelist, Linelist, Trafolist, Convlist, Loadlist, Genlist, Storlist, net, period_duration, slack_comp

[buses, base_power, Nodelist, Linelist, Trafolist, Convlist, Loadlist, Genlist, Storlist, net, period_duration, slack_comp] = Griddata(general_data,node_data,line_data,transformer_data,converter_data,load_data,gen_data,stor_data)


# Misses how to handle with empty lists, for exemple, in case that are not used transformers in network


In [ ]:
# Plotting the droop curves of each load and generator equipment 

# Load equipment
# print('\nPlotting the VP droop curve caracteristic of each load')

print(">Plot droop evolution")

time_list = [i for i in Loadlist[0].droop.columns if type(i) is int]

fig, axs = plt.subplots(len(time_list),1,figsize=(14, 30),layout='constrained')
#fig.title("Droop Evolution")
labels = []
for i in range(0,len(time_list)):
    for j in range(0,len(Loadlist)):
        auxV = Loadlist[j].droop.index[Loadlist[j].droop['Droop'].str.contains('V')]
        auxP = Loadlist[j].droop.index[Loadlist[j].droop['Droop'].str.contains(r'P\d+', regex=True)]   
        axs[i].plot(Loadlist[j].droop.loc[auxP,time_list[i]].values,Loadlist[j].droop.loc[auxV,time_list[i]].values, marker='o', linestyle='-')     
        labels.append(['Load' + str(Loadlist[j].load_id)])
    axs[i].set_ylabel("Voltage (pu)")
    axs[i].legend(labels)
    axs[i].grid(True)
    
plt.show()

# Generator equipment
# print('\nPlotting the VP droop curve caracteristic of each generator')

print(">Plot droop evolution")

fig, axs = plt.subplots(len(time_list),1,figsize=(14, 30),layout='constrained')
#fig.title("Droop Evolution")
labels = []
for i in range(0,len(time_list)):
    for j in range(0,len(Genlist)):
        auxV = Genlist[j].droop.index[Genlist[j].droop['Droop'].str.contains('V')]
        auxP = Genlist[j].droop.index[Genlist[j].droop['Droop'].str.contains(r'P\d+', regex=True)]   
        axs[i].plot(Genlist[j].droop.loc[auxP,time_list[i]].values,Genlist[j].droop.loc[auxV,time_list[i]].values, marker='o', linestyle='-')     
        labels.append(['Generator' + str(Genlist[j].gen_id)])
    axs[i].set_ylabel("Voltage (pu)")
    axs[i].legend(labels)
    axs[i].grid(True)
    
plt.show()



# Storage equipment
# print('\nPlotting the VP droop curve caracteristic of each storage')

print(">Plot droop evolution")

fig, axs = plt.subplots(len(time_list),1,figsize=(14, 30),layout='constrained')
#fig.title("Droop Evolution")
labels = []
for i in range(0,len(time_list)):
    for j in range(0,len(Storlist)):
        auxV = Storlist[j].droop.index[Storlist[j].droop['Droop'].str.contains('V')]
        auxP = Storlist[j].droop.index[Storlist[j].droop['Droop'].str.contains(r'P\d+', regex=True)]   
        axs[i].plot(Storlist[j].droop.loc[auxP,time_list[i]].values,Storlist[j].droop.loc[auxV,time_list[i]].values, marker='o', linestyle='-')     
        labels.append(['Storage' + str(Storlist[j].stor_id)])
    axs[i].set_ylabel("Voltage (pu)")
    axs[i].legend(labels)
    axs[i].grid(True)
    

plt.show()

In [ ]:
# Selection of Time-Variant or Steady-State PF

def timeQuest(tl): # This function allows to give the pretended time series that the user want to study

    tl_chosen = []

    PF_type = input('\nWhat type of Power Flow Analysis is pretended to make? For Time-Dependent, type TD, for Steady-State, type SS, to exit, EXIT')

    if PF_type == 'TD':
        tl_chosen = tl.copy()
    elif PF_type == 'SS':
        
        print('\nThe list of time instants that is present in the Excel is given by',tl) # Print the time data for user choose the time instant that want to study 
        usInput = input('\nWhat is the time(s) instant(s) that the user want to study?') # Receive the hourly data from user (for exemple [1,2,...] or 1 2 ...) and treat that data
        print(usInput)
        if len(usInput) > 1:
            aux_usInput = [tl[x] for x in range(0,len(tl)) if usInput in tl[x]]
        else:
            aux_usInput = usInput
        tl_chosen.append(aux_usInput)
    elif PF_type == 'EXIT':
        return
    else:
        print('\nCaution! The value introduced is not according to the expected input.')
        return timeQuest(tl)

    return tl_chosen

def loaddroopReadjustment(net,Loadlist,bus_voltages,base_power,aux_Matrixp,T,t):

    for i in range(0,len(Loadlist)): 
        
        auxV = Loadlist[i].droop.index[Loadlist[i].droop['Droop'].str.contains('V')]
        auxP = Loadlist[i].droop.index[Loadlist[i].droop['Droop'].str.contains(r'P\d+', regex=True)]
        auxPS = Loadlist[i].droop.index[Loadlist[i].droop['Droop'].str.contains(r'P[A-Za-z]+', regex=True)]
        auxQS = Loadlist[i].droop.index[Loadlist[i].droop['Droop'].str.contains(r'Q[A-Za-z]+', regex=True)]
        loadIdx = net.load.index[Loadlist[i].load_id==net.load['name']]

        for j in range(0,len(Nodelist)):
            if Nodelist[j].node_number.item() == Loadlist[i].bus:
                aux_electype = Nodelist[j].elec_type.item()

        if aux_electype == 'DC':

            if Loadlist[i].dp_cont == 1:

                pdroop = np.interp(bus_voltages[buses[Loadlist[i].bus]].item(),Loadlist[i].droop.loc[auxV,T].values,Loadlist[i].droop.loc[auxP,T].values)
                pset = Loadlist[i].droop.loc[auxPS,T].values
                pload = 0
            
                if abs(pset) > abs(pdroop): # Verification if setpoint of power is higher than droop curve
                    pload = pdroop
                else:
                    pload = pset
                
                net.load.loc[loadIdx,'p_mw'] = pload*Loadlist[i].pn/1000
            
            elif Loadlist[i].dp_cont == 0 and t == 0:

                net.load.loc[loadIdx,'p_mw']=(Loadlist[i].droop.loc[auxPS,T]).item()*Loadlist[i].pn/1000

        elif aux_electype == 'AC':

            net.load.loc[loadIdx,'p_mw']=(Loadlist[i].droop.loc[auxPS,T]).item()*Loadlist[i].pn/1000
            net.load.loc[loadIdx,'q_mvar']=(Loadlist[i].droop.loc[auxQS,T]).item()*Loadlist[i].pn/1000

        aux_Matrixp[i,t] = (float(net.load.loc[loadIdx,'p_mw'].iloc[0])*(1000/base_power.values)).item()

    return net,aux_Matrixp

def gendroopReadjustment(net,Genlist,bus_voltages,base_power,T):

    for i in range(0,len(Genlist)): 
    
        auxV = Genlist[i].droop.index[Genlist[i].droop['Droop'].str.contains('V')]
        auxP = Genlist[i].droop.index[Genlist[i].droop['Droop'].str.contains(r'P\d+', regex=True)]
        auxPS = Genlist[i].droop.index[Genlist[i].droop['Droop'].str.contains(r'P[A-Za-z]+', regex=True)]
        auxQS = Genlist[i].droop.index[Genlist[i].droop['Droop'].str.contains(r'Q[A-Za-z]+', regex=True)]
        genIdx = net.gen.index[Genlist[i].gen_id==net.gen['name']] 

        for j in range(0,len(Nodelist)):
            if Nodelist[j].node_number.item() == Genlist[i].bus:
                aux_electype = Nodelist[j].elec_type.item()

        if aux_electype == 'DC':

            if Genlist[i].dp_cont == 1:

                pdroop = np.interp(bus_voltages[buses[Genlist[i].bus]].item(),Genlist[i].droop.loc[auxV,T].values,Genlist[i].droop.loc[auxP,T].values)
                pset = Genlist[i].droop.loc[auxPS,T].values
                pgen = 0
            
                if abs(pset) > abs(pdroop): # Verification if setpoint of power is higher than droop curve
                    pgen = pdroop
                else:
                    pgen = pset
                
                if Genlist[i].gen_id == slack_comp.values:
                    net.gen.loc[genIdx,'p_mw'] = pgen*Genlist[i].pn/1000

                else:
                    net.sgen.loc[genIdx,'p_mw'] = pgen*Genlist[i].pn/1000
            
            elif Genlist[i].dp_cont == 0 and t == 0:

                if Genlist[i].gen_id == slack_comp.values:

                    net.gen.loc[genIdx,'p_mw'] = (Genlist[i].droop.loc[auxPS,T]).item()*Genlist[i].pn/1000

                else:
                    net.sgen.loc[genIdx,'p_mw'] = (Genlist[i].droop.loc[auxPS,T]).item()*Genlist[i].pn/1000

            else:

                if Genlist[i].gen_id == slack_comp.values:

                    net.gen.loc[genIdx,'p_mw'] = (Genlist[i].droop.loc[auxPS,T]).item()*Genlist[i].pn/1000

                else:
                    net.sgen.loc[genIdx,'p_mw'] = (Genlist[i].droop.loc[auxPS,T]).item()*Genlist[i].pn/1000


        elif aux_electype == 'AC':
            
            if Genlist[i].gen_id == slack_comp.values:
                net.gen.loc[genIdx,'p_mw']=(Genlist[i].droop.loc[auxPS,T]).item()*Genlist[i].pn/1000
                net.gen.loc[genIdx,'q_mvar']=(Genlist[i].droop.loc[auxQS,T]).item()*Genlist[i].pn/1000

            else:
                net.gen.loc[genIdx,'p_mw']=(Genlist[i].droop.loc[auxPS,T]).item()*Genlist[i].pn/1000
                net.gen.loc[genIdx,'q_mvar']=(Genlist[i].droop.loc[auxQS,T]).item()*Genlist[i].pn/1000

    return net

def storControl(net,Storlist,bus_voltages,base_power,period_duration,aux_Matrixsoc,T,t): 

    for i in range(0,len(Storlist)):

        auxV = Storlist[i].droop.index[Storlist[i].droop['Droop'].str.contains('V')]
        auxP = Storlist[i].droop.index[Storlist[i].droop['Droop'].str.contains(r'P\d+', regex=True)]
        auxPS = Storlist[i].droop.index[Storlist[i].droop['Droop'].str.contains(r'P[A-Za-z]+', regex=True)]
        auxQS = Storlist[i].droop.index[Storlist[i].droop['Droop'].str.contains(r'Q[A-Za-z]+', regex=True)]
        storIdx = net.storage.index[Storlist[i].stor_id==net.storage['name']]
        P_stor = 0
        Q_stor = 0

        for j in range(0,len(Nodelist)):
            if Nodelist[j].node_number.item() == Storlist[i].bus:
                aux_electype = Nodelist[j].elec_type.item()

        if aux_electype == 'DC':

            if Storlist[i].dp_cont == 1:

                pdroop = np.interp(bus_voltages[buses[Storlist[i].bus]].item(),Storlist[i].droop.loc[auxV,T].values,Storlist[i].droop.loc[auxP,T].values)
                pset = Storlist[i].droop.loc[auxPS,T].values
                pstor = 0
            
                if abs(pset) > abs(pdroop): # Verification if setpoint of power is higher than droop curve
                    pstor = pdroop
                else:
                    pstor = pset
                
                net.storage.loc[storIdx,'p_mw'] = pstor*Storlist[i].pn/1000
            
            elif Storlist[i].dp_cont == 0 and t == 0:

                net.storage.loc[storIdx,'p_mw']=(Storlist[i].droop.loc[auxPS,T])*Storlist[i].pn/1000
            
            P_stor = net.storage.loc[storIdx,'p_mw'].item()
            Q_stor = 0

        elif aux_electype == 'AC':

            net.storage.loc[storIdx,'p_mw']=(Storlist[i].droop.loc[auxPS,T])*Storlist[i].pn/1000
            net.storage.loc[storIdx,'q_mvar']=(Storlist[i].droop.loc[auxQS,T])*Storlist[i].pn/1000

            P_stor = net.storage.loc[storIdx,'p_mw'].item()
            Q_stor = net.storage.loc[storIdx,'q_mvar'].item()

        S_stor = mt.sqrt(P_stor**2+Q_stor**2)

        SOC_ini_var = (S_stor*period_duration.item())/Storlist[i].max_ener

        SOC_f = SOC_ini_var + Storlist[i].ini_SOC

        if SOC_f < Storlist[i].min_SOC: # SOC lower limit verification 

            SOC_max_var = Storlist[i].ini_SOC - Storlist[i].min_SOC
            net.storage.loc[storIdx,'p_mw'] = (SOC_max_var*Storlist[i].max_ener/period_duration.item())*Storlist[i].pn/1000

        elif SOC_f > Storlist[i].max_SOC: # SOC upper limit verification

            SOC_max_var = Storlist[i].max_SOC - Storlist[i].ini_SOC
            net.storage.loc[storIdx,'p_mw'] = (SOC_max_var*Storlist[i].max_ener/period_duration.item())*Storlist[i].pn/1000

        Storlist[i].ini_SOC = SOC_f

        net.storage.loc[storIdx,'soc_percent'] = Storlist[i].ini_SOC        

        aux_Matrixsoc[i,t] = net.storage.loc[storIdx,'soc_percent'].item()

    return net,aux_Matrixsoc

def convEst(net,Nodelist,Convlist,bus_voltages,bus_voltages_angles,base_power,tol_max_perc): # In progress

    for i in range(0,len(Convlist)):

        if Convlist[i].convtype == 'Converter DC/DC':

            convIdx = net.trafo.index[('Converter DC/DC ' + str(Convlist[i].conv_id))==net.trafo['name']].item()

            if (bus_voltages[buses[Convlist[i].from_bus]]-bus_voltages[buses[Convlist[i].to_bus]]) == 0:

                Req_corr = 0.005*(Convlist[i].pn/base_power.item())

            else:

                Pconv = (bus_voltages[buses[Convlist[i].from_bus]]-bus_voltages[buses[Convlist[i].to_bus]])**2/net.trafo.loc[convIdx,'vk_percent']
                
                aux_PF = Convlist[i].eff_table['Efficiency Curve'].dropna().unique()
                auxidxEC = Convlist[i].eff_table.columns.get_loc('Efficiency Curve')
                aux_PO = Convlist[i].eff_table.columns[auxidxEC+1:].values
                aux_eff = Convlist[i].eff_table.iloc[:,auxidxEC+1:]

                points = [(fp, p) for fp in aux_PF for p in aux_PO]
                values = aux_eff.values.flatten()

                fp_novo, p_novo = 1, Pconv

                eff = griddata(points, values, (fp_novo, p_novo), method='linear')

                Req = bus_voltages[buses[Convlist[i].from_bus]]*(bus_voltages[buses[Convlist[i].from_bus]]-bus_voltages[buses[Convlist[i].to_bus]])/(net.res_bus.p_mw[buses[Convlist[i].from_bus]].item()/(base_power.item()*1000))

                if abs(Req-(((bus_voltages[buses[Convlist[i].from_bus]].item()-bus_voltages[buses[Convlist[i].to_bus]].item())**2)/((1-eff)*(net.res_bus.p_mw[buses[Convlist[i].from_bus]]/(base_power.item()*1000)))))>tol_max_perc:

                    Req_corr = (((bus_voltages[buses[Convlist[i].from_bus]].item()-bus_voltages[buses[Convlist[i].to_bus]].item())**2)/((1-eff)*(net.res_bus.p_mw[buses[Convlist[i].from_bus]]/(base_power.item()*1000)))) # In grid pu
                
                else:

                    Req_corr = Req

            Req_corr = Req_corr * (base_power.item()/Convlist[i].pn)

            net.trafo.loc[convIdx,'vk_percent'] = 1.001*Req_corr

            net.trafo.loc[convIdx,'vkr_percent'] = Req_corr
        
        if Convlist[i].convtype == 'Converter AC/DC': 

            conv_trafoIdx = net.trafo.index[('Converter AC/DC ' + str(Convlist[i].conv_id))==net.trafo['name']].item() # Index of the converter in transformer list in net data

            conv_wardIdx = net.ward.index[('Converter AC/DC ' + str(Convlist[i].conv_id))==net.ward['name']].item() # Index of the converter in ward list in net data
            
            bus_comp_volt = []

            k = 1.01    # Constant just to make the ratio between the resistance and the reactance of the transformer model in order to allow the convergence of the power flow

            for j in range(0,len(bus_voltages)):
                bus_comp_volt.append(complex(bus_voltages[j]*mt.cos(bus_voltages_angles[j]),bus_voltages[j]*mt.sin(bus_voltages_angles[j])))    # Obtaining the complex voltages of each bus

            if all(i == 1 for i in bus_comp_volt):
                
                Ractual = 1e-3                                  # Actual resistance (pu grid)
                absZactual = k*Ractual
                Xactual = mt.sqrt(absZactual**2-Ractual**2)     # Actual reactance (pu grid)
                Req_corr = Ractual
                Xeq_corr = Xactual
                ps_conv = 0

            else:
                
                # Actualized iteration converter parameters
                 
                absZactual = (net.trafo.loc[conv_trafoIdx,'vk_percent']/100) * (base_power.item()/(Convlist[i].pn)) / (1e3)     # Actual impedance (pu grid)    (1e3 due to Pandapower works in MVA instead of kVA)
                Ractual = (net.trafo.loc[conv_trafoIdx,'vkr_percent']/100) * (base_power.item()/(Convlist[i].pn)) / (1e3)       # Actual resistance (pu grid)   
                Xactual = mt.sqrt(absZactual**2-Ractual**2)                                                                     # Actual reactance (pu grid)
                Zactual = complex(Ractual,Xactual)
                Yactual = 1/Zactual
                ps_actual = net.trafo.loc[conv_trafoIdx,'shift_degree'] * (mt.pi/180)                                           # Phase-shift parameter in transformer converter equivalent model (Already save in a variable in case that if needed)
                m = 1                                                                                                           # We assume for now a constant ratio between voltages 
                m_complex_actual = complex(m*np.cos(ps_actual),m*np.sin(ps_actual))
                Qs_ward_actual = net.ward.loc[conv_wardIdx,'qs_mvar']

                # Verification of lv, hv, AC and DC buses and their voltages of the 2 elements converter model

                def verf_buses(net,conv_trafoIdx,conv_wardIdx):

                    hv_bus = net.trafo.loc[conv_trafoIdx,'hv_bus'].item()
                    lv_bus = net.trafo.loc[conv_trafoIdx,'lv_bus'].item()
                    AC_bus = net.ward.loc[conv_wardIdx,'bus'].item()
                    
                    if AC_bus==hv_bus:
                        DC_bus = lv_bus
                    else:
                        DC_bus = hv_bus

                    return hv_bus, lv_bus, AC_bus, DC_bus

                # Function for computation of Power Flows of Pandapower Transformer model (HV side to LV side and vice-versa)

                def basic_power_flow_eqs(hv_bus,lv_bus,V_hv,V_lv,Yactual,m_complex_actual):

                    S_hv_lv = (Yactual.conjugate()/(abs(m_complex_actual)**2))*(abs(V_hv)**2-m_complex_actual.conjugate()*V_hv*V_lv.conjugate())                    # Complex Power flowing from HV side to LV side 
                    S_lv_hv = (Yactual.conjugate()/(abs(m_complex_actual)**2))*((abs(V_lv)**2)*(abs(m_complex_actual)**2)-m_complex_actual*V_lv*V_hv.conjugate())   # Complex Power flowing from LV side to HV side

                    S_losses = S_hv_lv + S_lv_hv
                    P_losses = S_losses.real
                    Q_losses = S_losses.imag

                    return S_hv_lv, S_lv_hv, P_losses, Q_losses

                # Function for verification of power flow (if power flows from AC to DC or vice-versa)

                def verf_power_flow(S_hv_lv,S_lv_hv):

                    if abs(S_hv_lv.real) > abs(S_lv_hv.real):
                        Pout = S_lv_hv.real
                        Qout = S_lv_hv.imag
                        Sout = complex(Pout,Qout)
                        FPout = mt.cos(abs(Pout/abs(Sout)))
                        Pin = S_hv_lv.real
                        Qin = S_hv_lv.imag
                        Sin = complex(Pin,Qin)
                        FPin = mt.cos(abs(Pin/abs(Sin)))
                        eff_actual = abs(Pout/Pin)

                    else:
                        Pout = S_hv_lv.real
                        Qout = S_hv_lv.imag
                        Sout = complex(Pout,Qout)
                        FPout = mt.cos(abs(Pout/abs(Sout)))
                        Pin = S_lv_hv.real
                        Qin = S_lv_hv.imag
                        Sin = complex(Pin,Qin)
                        FPin = mt.cos(abs(Pin/abs(Sin)))
                        eff_actual = abs(Pout/Pin)
                        
                    return Sout,FPout,Sin

                # Estimation of efficiency from converter data

                def effest(Pout,FPOut,aux_PF,aux_PO,aux_eff):

                    points = [(fp, p) for fp in aux_PF for p in aux_PO]
                    values = aux_eff.values.flatten()

                    fp_novo, p_novo = FPOut, Pout

                    eff = griddata(points, values, (fp_novo, p_novo), method='linear')

                    return eff

                ######## DECISION/CONTROL MODULE OF CONVERTER ########

                # Preliminary computations 

                hv_bus, lv_bus, AC_bus, DC_bus = verf_buses(net,conv_trafoIdx,conv_wardIdx)
                V_hv = bus_comp_volt[hv_bus]
                V_lv = bus_comp_volt[lv_bus]
                V_AC = bus_comp_volt[AC_bus]
                V_DC = bus_comp_volt[DC_bus]
                S_hv_lv, S_lv_hv, P_losses, Q_losses = basic_power_flow_eqs(hv_bus,lv_bus,V_hv,V_lv,Yactual,m_complex_actual)
                Sout, FPout, Sin = verf_power_flow(S_hv_lv,S_lv_hv)

                # Auxiliary vectors and indexes for the determination of the point of efficiency of the converter

                aux_PF = Convlist[i].eff_table['Efficiency Curve'].dropna().unique()
                auxidxEC = Convlist[i].eff_table.columns.get_loc('Efficiency Curve')
                aux_PO = Convlist[i].eff_table.columns[auxidxEC+1:].values
                aux_eff = Convlist[i].eff_table.iloc[:,auxidxEC+1:]

                # Point of operation interpolated from efficiency table from converter data

                Pout = abs(Sout.real)
                Pin = abs(Sin.real)
                eff_actual = effest(Pout*(base_power.item()/(Convlist[i].pn)),FPout,aux_PF,aux_PO,aux_eff)

                # Control module

                # Verification functions (In progress)

                if eff_actual != Pout/Pin:  # Function to verify the efficiency of pandapower vs efficiency table 
                    
                    Gt = (Pin * (1-eff_actual))/(((abs(V_AC)**2+(abs(m_complex_actual)**2)*abs(V_DC)**2)/abs(m_complex_actual)**2)+(((2*abs(V_AC)*abs(V_DC))/abs(m_complex_actual))*np.cos(np.angle(V_AC)-np.angle(V_DC)-np.angle(m_complex_actual))))

                    print(Pin * (1-eff_actual),P_losses,((abs(V_AC)**2+(abs(m_complex_actual)**2)*abs(V_DC)**2)/abs(m_complex_actual)**2)+(((2*abs(V_AC)*abs(V_DC))/abs(m_complex_actual))*np.cos(np.angle(V_AC)-np.angle(V_DC)-np.angle(m_complex_actual))))

                    Ractual = 1/(Gt*(k**2))

                    Xactual = Ractual*mt.sqrt(k**2-1)

                if abs(S_hv_lv.imag) > abs(S_lv_hv.imag) and abs(S_hv_lv.imag)>1e-6:    # Function to verify the reactive power flow in branch of transformer model and compensate it   

                    Qs_ward_actual = S_hv_lv.imag

                elif abs(S_lv_hv.imag) > abs(S_hv_lv.imag) and abs(S_lv_hv.imag)>1e-6:

                    Qs_ward_actual = S_lv_hv.imag

                else:

                    Qs_ward_actual = Qs_ward_actual
                     
                # Control function (To implement a function that allows the output voltage or power to be controlled by a reference value for exemple a imposed voltage output or imposed power output) (In progress)
                   

                Req_corr = Ractual
                Xeq_corr = Xactual
                ps_conv = ps_actual * (180/mt.pi)
                Qs_ward_corr = Qs_ward_actual

                print('\nR = ',Req_corr,'\nX = ',Xeq_corr,'\nEficiencia atual= ',eff_actual)


            Req_corr = Req_corr * (Convlist[i].pn/base_power.item())

            Xeq_corr = Xeq_corr * (Convlist[i].pn/base_power.item())

            net.trafo.loc[conv_trafoIdx,'vk_percent'] = mt.sqrt(Req_corr**2+Xeq_corr**2) * 100

            net.trafo.loc[conv_trafoIdx,'vkr_percent'] = Req_corr * 100

            net.trafo.loc[conv_trafoIdx,'shift_degree'] = ps_conv

    return net


def runpptd_cusm(net,buses,Nodelist,Linelist,Trafolist,Convlist,Loadlist,Genlist,Storlist,base_power):

    time_list_chosen = timeQuest(time_list) # List of time instant(s) chosen by the user 

    aux_forloop = False # Auxiliary variable to know if the for loop of T is running as the first time (assumes a PF start as a 'flat start') or have previous time values
                        # Since the previous solution has already set the power values to loads, in the run of the next time iteration is not needed to prepare the values using the new VP droop curves

    tol_max_perc = 1e-8 # Maximum tolerance in percentage

    n_iter_max = 10000 # Maximum number of iterations assumed

    n_iter_PF = 10 # Maximum number of iteration of Pandapower PF 

    erro = 1 # Error

    bus_voltages_old_print = [np.zeros([len(Nodelist),n_iter_max]) for i in time_list_chosen] # Just for printing bus voltages behaviors with the evolution of the iterations

    load_p_old_print = [np.zeros([len(Loadlist),n_iter_max]) for i in time_list_chosen]         # Just for printing how the power in each load behaves in each iteration and in each time iteration

    SOC_old_print = [np.zeros([len(Storlist),n_iter_max]) for i in time_list_chosen]         # Just for printing how the SOC in each BESS behaves in each iteration and in each time iteration

    aux_Matrix = np.ones([len(Nodelist),n_iter_max])    # Auxiliary matrix for bus voltages 

    aux_Matrixp = np.ones([len(Loadlist),n_iter_max])   # Auxiliary matrix for loads power

    aux_Matrixsoc = np.ones([len(Storlist),n_iter_max]) # Auxiliary matrix for SOC 

    start_time_TD = tm.time()

    for T in time_list_chosen:

        T = int(T)

        # Run power flow analysis
        print(">Run power flow analysis")

        repetir = True # Condition to continue while loop until the PF analysis is not completed

        t = 0 # Iteration counter

        # Droop preparation

        bus_voltages_old = np.zeros(len(Nodelist))         # Auxiliary array to memorize the results for comparison them between iterations 
        bus_power_injections_old = np.zeros(len(Nodelist))         # Auxiliary array to memorize the results for comparison them between iterations 

        for i in range(0,len(Nodelist)):
        
            aux_Matrix[buses[Nodelist[i].node_number.item()],t] = 1               # Define the values of the first iteration for printing 
        
        if aux_forloop is False:                        # Droop adjustment or converters state estimation for initial time step 

            # Converter equipment

            net = convEst(net,Nodelist,Convlist,np.ones(len(Nodelist)),np.zeros(len(Nodelist)),base_power,tol_max_perc)
            
            # Load equipment 

            [net,aux_Matrixp] = loaddroopReadjustment(net,Loadlist,np.ones(len(Nodelist)),base_power,aux_Matrixp,T,t)

            # Generator equipment

            net = gendroopReadjustment(net,Genlist,np.ones(len(Nodelist)),base_power,T)

            # Storage equipment

            [net,aux_Matrixsoc] = storControl(net,Storlist,np.ones(len(Nodelist)),base_power,period_duration,aux_Matrixsoc,T,t)

            aux_forloop = True

        start_time = tm.time()

        while (repetir is True):

            ###### PowerFlow analysis procedure ######

            try:
                pp.runpp(net,max_iteration=n_iter_PF)
                print(net.res_bus)
            except pp.LoadflowNotConverged:
                print("Power Flow did not converge")

            bus_indices = net.res_bus.index + 1     # adjust to start from 1
            bus_voltages = net.res_bus.vm_pu
            bus_voltages_angles = net.res_bus.va_degree * (mt.pi/180)
            bus_power_injections = net.res_bus.p_mw
            
            desvio = abs(bus_voltages-bus_voltages_old)/bus_voltages + abs(bus_power_injections-bus_power_injections_old)/bus_power_injections # Deviation measurement of voltage profile of each iteration and active power injections (in percentage) 

            erro = (max(desvio))*100 # Error computation of the higher deviation in percentage

            if (erro < tol_max_perc) and t!=0:               # Comparison between iterations resorting to error
                end_time = tm.time()
                elapsed_time = end_time - start_time
                print("Power Flow solution obtained with an error of",erro,"% at",t+1,"iterations in",elapsed_time*1000,"ms for time instant",T)
                repetir = False

            elif (t>=n_iter_max):
                print("Power Flow solution not obtained due to maximum number of iterations being reached with an error of",erro,"%")
                repetir = False

            bus_voltages_old = bus_voltages.values
            bus_power_injections_old = bus_power_injections.values

            ###### Control Functions - Converter Equivalent Resistance Estimation and Droop Readjusment ######
            
            # Converter equipment

            net = convEst(net,Nodelist,Convlist,bus_voltages,bus_voltages_angles,base_power,tol_max_perc)

            # Load equipment

            [net,aux_Matrixp] = loaddroopReadjustment(net,Loadlist,bus_voltages,base_power,aux_Matrixp,T,t)
                
            # Generator equipment

            net = gendroopReadjustment(net,Genlist,bus_voltages,base_power,T)

            # Storage equipment

            [net,aux_Matrixsoc] = storControl(net,Storlist,bus_voltages,base_power,period_duration,aux_Matrixsoc,T,t)

            t=t+1 # Iteration counter

            # Define the values for the atual iteration for printing

            for i in range(0,len(Nodelist)):

                aux_Matrix[buses[Nodelist[i].node_number.item()],t] = bus_voltages_old[buses[Nodelist[i].node_number.item()]]        # Define the values of the first iteration for printing 

            # Remove the remaining elements that does not count for printing purposes

            if (repetir == False):
                
                aux_Matrix = np.delete(aux_Matrix,range(t,n_iter_max),1) 
                aux_Matrixp = np.delete(aux_Matrixp,range(t,n_iter_max),1)
                aux_Matrixsoc = np.delete(aux_Matrixsoc,range(t,n_iter_max),1)
        
        if len(time_list_chosen)>1:

            bus_voltages_old_print[T-1] = aux_Matrix      # Since the index might be not exact with the time instant for exemple (time instant chosen of 2 and 3, so first index is not 2), this must be improved
            load_p_old_print[T-1] = aux_Matrixp
            SOC_old_print[T-1] = aux_Matrixsoc

        else:

            bus_voltages_old_print = aux_Matrix
            load_p_old_print = aux_Matrixp
            SOC_old_print = aux_Matrixsoc

        del aux_Matrix,aux_Matrixp,aux_Matrixsoc

        aux_Matrix = np.zeros([len(Nodelist),n_iter_max])       # It is needed to resize the aux_Matrix since the loop is going to go to another time instant
        aux_Matrixp = np.zeros([len(Loadlist),n_iter_max])      # It is needed to resize the aux_Matrixp since the loop is going to go to another time instant
        aux_Matrixsoc = np.zeros([len(Storlist),n_iter_max])    # It is needed to resize the aux_Matrixsoc since the loop is going to go to another time instant

    end_time_TD = tm.time()
    elapsed_time_TD = end_time_TD - start_time_TD

    print("Power Flow solution obtained in",elapsed_time_TD*1000,"ms for time interval between",min(time_list_chosen),'and',max(time_list_chosen))

    return net,bus_voltages_old_print,load_p_old_print,SOC_old_print,bus_indices,bus_voltages,bus_voltages_angles

[net,bus_voltages_old_print,load_p_old_print,SOC_old_print,bus_indices,bus_voltages,bus_voltages_angles] = runpptd_cusm(net,buses,Nodelist,Linelist,Trafolist,Convlist,Loadlist,Genlist,Storlist,base_power)



In [ ]:
# Print results
print(">Print results")
print(net.res_bus)
print(net.res_line)
print(net.res_storage)
print(net.res_trafo)
# print("Bus voltages (pu):", net.res_bus.vm_pu)
# print("Line loading (A):", net.res_line.loading_percent)

In [ ]:
# Plot voltage profiles

def printVoltProf(bus_indices,bus_voltages_old_print):
    print(">Plot voltage profiles")

    fig, axs = plt.subplots(len(bus_voltages_old_print),1,figsize=(14, 30),layout='constrained')
    title_names = []

    for i in range(0,len(bus_voltages_old_print)):       
        aux_data = bus_voltages_old_print[i]
        if aux_data.shape[1]>1:
            axs[i].plot(bus_indices,aux_data[:,-1])
        else:
            axs[i].plot(bus_indices,aux_data[:])
        title_names = ['Time Instant = ' + str(i + 1)]
        axs[i].set_title(title_names)
        axs[i].set_ylabel("Voltage (pu)")
        axs[i].grid(True)


    #plt.figure(figsize=(8.5, 2.25))
    #plt.plot(bus_indices, bus_voltages_old_print, marker='o', linestyle='-', color='b')
    #plt.title("Voltage Profiles")
    #plt.xlabel("Bus Index")
    #plt.ylabel("Voltage (pu)")
    #plt.grid(True)
    # plt.ylim(0.9, 1.1)
    # plt.ylim(0.98, 1.06)
    #plt.xticks(bus_indices)
    plt.show()
    return

printVoltProf(bus_indices,bus_voltages_old_print)

In [ ]:
# Plot bus voltage evolution in iterations by each bus 

def printVoltIter(bus_voltages_old_print):
    print(">Plot bus voltages vs iterations evolution")

    if type(bus_voltages_old_print) is list:
        
        auxPrint = 0
        auxIdx = []
        
        for i in range(0,len(bus_voltages_old_print)):
            
            if bus_voltages_old_print[i].shape[1] > 1:
                
                auxPrint += 1
                auxIdx.append(i)


        fig, axs = plt.subplots(auxPrint,1,figsize=(14, 30),layout='constrained')
        title_names = []

        for i in range(0,auxPrint):
                
                axs[i].plot(bus_voltages_old_print[auxIdx[i]].transpose())
                title_names = ['Time Instant = ' + str(auxIdx[i] + 1)]
                axs[i].set_title(title_names)
                axs[i].set_ylabel("Voltage (pu)")
                axs[i].set_xlim(0, bus_voltages_old_print[auxIdx[i]].shape[1]-1)
                axs[i].set_xticks(range(1,bus_voltages_old_print[auxIdx[i]].shape[1]))
                axs[i].grid(True)
        
        plt.xlabel("Iterations")
        plt.show

    else:

        plt.plot(bus_voltages_old_print.transpose())
        plt.show

    return

def printPowerIter(load_p_old_print):
    
    print(">Plot power components vs iterations evolution")

    if type(load_p_old_print) is list:
        
        auxPrint = 0
        auxIdx = []
        
        for i in range(0,len(load_p_old_print)):
            
            if load_p_old_print[i].shape[1] > 1:
                
                auxPrint += 1
                auxIdx.append(i)


        fig, axs = plt.subplots(auxPrint,1,figsize=(14, 30),layout='constrained')
        title_names = []

        for i in range(0,auxPrint):
                
                axs[i].plot(load_p_old_print[auxIdx[i]].transpose())
                title_names = ['Time Instant = ' + str(auxIdx[i] + 1)]
                axs[i].set_title(title_names)
                axs[i].set_ylabel("Power (pu)")
                axs[i].set_xlim(0, load_p_old_print[auxIdx[i]].shape[1]-1)
                axs[i].set_xticks(range(1,load_p_old_print[auxIdx[i]].shape[1]))
                axs[i].grid(True)
        
        plt.xlabel("Iterations")
        plt.show

    else:

        plt.plot(load_p_old_print.transpose())
        plt.show

    return

def printSOCIter(SOC_old_print):
    
    print(">Plot SOC vs iterations evolution")

    if type(SOC_old_print) is list:
        
        auxPrint = 0
        auxIdx = []
        
        for i in range(0,len(SOC_old_print)):
            
            if SOC_old_print[i].shape[1] > 1:
                
                auxPrint += 1
                auxIdx.append(i)


        fig, axs = plt.subplots(auxPrint,1,figsize=(14, 30),layout='constrained')
        title_names = []

        for i in range(0,auxPrint):
                
                axs[i].plot(SOC_old_print[auxIdx[i]].transpose())
                title_names = ['Time Instant = ' + str(auxIdx[i] + 1)]
                axs[i].set_title(title_names)
                axs[i].set_ylabel("SOC (%)")
                axs[i].set_xlim(0, SOC_old_print[auxIdx[i]].shape[1]-1)
                axs[i].set_xticks(range(1,SOC_old_print[auxIdx[i]].shape[1]))
                axs[i].grid(True)
        
        plt.xlabel("Iterations")
        plt.show

    else:

        plt.plot(SOC_old_print.transpose())
        plt.show

    return

printVoltIter(bus_voltages_old_print)

printPowerIter(load_p_old_print)

printSOCIter(SOC_old_print)

''' Code written that could help

fig, axs = plt.subplots(len(drp_idx_aux),1,figsize=(14, 30),layout='constrained')
#fig.title("Droop Evolution")
labels = []
for i,_ in enumerate(droop_data.Node_j):
    axs[i].plot(range(0,t), bus_voltages_old_print[:,i], marker='o', linestyle='-')
    labels = [net.bus['name'].iloc[int(drp_idx_aux[i])]+' - PF']
    axs[i].set_ylabel("Voltage (pu)")
    axs[i].legend(labels)
    axs[i].grid(True)
    axs[i].set_xlim(0, t)
    #axs[i].set_ylim(bus_voltages_old_print[t-1,i]*0.995, bus_voltages_old_print[t-1,i]*1.005)
    #plt.plot(range(1,t+1), volt_aux.values[i,:], marker='+', linestyle=':')    
    #labels.append([net.bus['name'].iloc[int(drp_idx_aux[i])]+' - Power Voltage Droop Curve'])
plt.xlabel("Iterations")

labels = []
#plt.ylim(0.925, 1.075)
plt.show()

print(">Plot droop evolution")

plt.figure(figsize=(14, 6))
plt.title("Droop Evolution")
labels = []
for i,_ in enumerate(droop_data.Node_j):
    plt.plot(load_p_old_print[:,i],bus_voltages_old_print[:,i], marker='o', linestyle='-')
    labels.append([net.bus['name'].iloc[int(drp_idx_aux[i])]+' - PF'])
    plt.plot(p_aux.values[i,:], volt_aux.values[i,:], marker='+', linestyle=':')    
    labels.append([net.bus['name'].iloc[int(drp_idx_aux[i])]+' - Power Voltage Droop Curve'])
plt.legend(labels)
plt.xlabel("Power (pu)")
plt.ylabel("Voltage (pu)")
plt.grid(True)
#plt.ylim(0.925, 1.075)
#plt.xlim(-0.05, 2.1)
# plt.xticks(bus_indices)
plt.show()

'''

In [ ]:
# Computation of Environmental KPIs

# PUE - Power Usage Effectiveness (by load component)

